In [2]:
import torch
import torch.nn as nn

torch.manual_seed(77)

z = torch.randn(5) # z = f(x)
y_hat = torch.sigmoid(z) # y_hat = sigmoid(f(x))
y = torch.tensor([0.,1.,1.,0.,1.]) 

print(f"z = {z}\ny_hat = {y_hat}\ny = {y}\n{'':-^80}")

# Negative Log-likelihoods
loss_NLL_scratch = -(y * y_hat.log() + (1 - y) * (1 - y_hat).log())
print(f"Negative Log-likelihoods\n    {loss_NLL_scratch}")
print(f"Loss Summantion : {loss_NLL_scratch.sum()}")

z = tensor([-0.3568,  0.6007, -0.6968, -0.5242,  0.9087])
y_hat = tensor([0.4117, 0.6458, 0.3325, 0.3719, 0.7127])
y = tensor([0., 1., 1., 0., 1.])
--------------------------------------------------------------------------------
Negative Log-likelihoods
    tensor([0.5306, 0.4372, 1.1010, 0.4650, 0.3387])
Loss Summantion : 2.8724989891052246


In [2]:
import torch
import torch.nn as nn

torch.manual_seed(77)

### Multi class Setting ###

print(f"{'Setting up multiclass case':-^80}") 

z = torch.randn(5,3) # z = f(x)
y_hat = torch.softmax(z, dim=1) # y_hat = softmax(f(x))

y = torch.tensor([0,1,0,2,1]) 

print(f"z = {z}\ny_hat = {y_hat}\ny = {y}\n{'':-^80}")

# Negative Log-likelihoods 
loss_NLL_scratch = -y_hat.log()[torch.arange(5),y.long()] 
print(f"Negative Log-likelihoods\n    {loss_NLL_scratch}")
print(f"Loss Summantion : {loss_NLL_scratch.sum()}")

c:\Users\AY\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---------------------------Setting up multiclass case---------------------------
z = tensor([[-0.3568,  0.6007, -0.6968],
        [-0.5242,  0.9087, -1.6423],
        [ 0.4583, -0.1266,  0.2302],
        [ 0.0024, -0.8097,  1.3568],
        [-0.6798, -0.0881, -1.2044]])
y_hat = tensor([[0.2316, 0.6035, 0.1649],
        [0.1812, 0.7595, 0.0592],
        [0.4250, 0.2368, 0.3383],
        [0.1880, 0.0835, 0.7285],
        [0.2942, 0.5317, 0.1741]])
y = tensor([0, 1, 0, 2, 1])
--------------------------------------------------------------------------------
Negative Log-likelihoods
    tensor([1.4626, 0.2751, 0.8558, 0.3168, 0.6317])
Loss Summantion : 3.5419459342956543


# 오토인코더

In [3]:
# 입력 데이터의 열 수
num_features = 180
threshold = 0.1  # 이상치 판단 기준값

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # 인코더
        self.encoder = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )
        
        # 디코더
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, num_features),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded